## torch.backends.cudnn.benchmark  

大家在训练深度学习模型的时候，经常会使用 `GPU` 来加速网络的训练。但是说起 `torch.backends.cudnn.benchmark` 这个 `GPU` 相关的 `flag`，可能有人会感到比较陌生。在一般场景下，只要简单地在 `PyTorch` 程序开头将其值设置为 `True`，就可以大大提升卷积神经网络的运行速度。既然如此神奇，为什么 `PyTorch` 不将其默认设置为 `True`？它的适用场景是什么？为什么使用它可以提升效率？答案就在本文之中。

设置 `torch.backends.cudnn.benchmark=True` 将会让程序在开始时花费一点额外时间，为整个网络的每个卷积层搜索最适合它的卷积实现算法，进而实现网络的加速。适用场景是网络结构固定（不是动态变化的），网络的输入形状（包括 `batch size`，图片大小，输入的通道）是不变的，其实也就是一般情况下都比较适用。反之，如果卷积层的设置一直变化，将会导致程序不停地做优化，反而会耗费更多的时间。

`cuDNN` 是英伟达专门为深度神经网络所开发出来的 `GPU` 加速库，针对卷积、池化等等常见操作做了非常多的底层优化，比一般的 `GPU` 程序要快很多。大多数主流深度学习框架都支持 `cuDNN，PyTorch` 自然也不例外。在使用 `GPU` 的时候，`PyTorch` 会默认使用 `cuDNN` 加速。但是，在使用 `cuDNN` 的时候，`torch.backends.cudnn.benchmark` 模式是为 `False`。所以就意味着，我们的程序可能还可以继续提速！

In [1]:
import torch

In [3]:
torch.manual_seed(seed=0)

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='GeForce RTX 2080 Ti', major=7, minor=5, total_memory=10988MB, multi_processor_count=68)

In [7]:
torch.cuda.get_device_properties(0).name

'GeForce RTX 2080 Ti'

In [10]:
torch.cuda.get_device_properties(0).total_memory / 1024 ** 2  #bytes to MB

10988.125

###  torch.cuda.synchronize() 
 

In [ ]:
# one
start = time.time()
result = model(input)
end = time.time()

In [ ]:
# two
torch.cuda.synchronize()
start = time.time()
result = model(input)
torch.cuda.synchronize()
end = time.time()

一共上述两种测试时间的方式，正确的方式是第二种.在`pytorch`里面，程序的执行都是**异步**的。如果采用第一种方式，测试的时间会很短，因为执行完`end=time.time()`程序就退出了，后台的`cu`也因为`python`的退出退出了，如果采用第二种方式，代码会同步`cu`的操作，等待`gpu`上的操作都完成了再继续成形`end = time.time()`  
如果将第一段代码改为

In [ ]:
start = time.time()
result = model(input)
print(result)
end = time.time()

这时候会发现第三段代码和第二段代码的时间是类似的，因为第三段代码会等待`gpu`上的结果执行完传给`print`函数，所以整个时间就和第二段同步的操作的时间基本上是一致的，将`print(result)`换成`result.cpu()`结果是一致的。

### thop(计算模型flops)  

`FLOPS`：注意全大写，是`floating point operations per second`的缩写，意指每秒浮点运算次数，理解为计算速度。是一个衡量硬件性能的指标。  
`FLOPs`：注意s小写，是`floating point operations`的缩写（s表复数），意指浮点运算数，理解为计算量。可以用来衡量算法/模型的复杂度。网上打字很容易全小写，造成混淆，本问题针对模型，应指的是`FLOPs`。

`Paper`里比较流行的单位是`GFLOPs`:`1 GFLOPs = 10^9 FLOPs = 2 * Mac`即：10亿次浮点运算  


In [2]:
!pip install thop

In [3]:
import thop
class YourModule(nn.Module):   
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, )

计算FLOPs的代码和包，即傻瓜又好用  

In [7]:
!pip install ptflops

  Created wheel for ptflops: filename=ptflops-0.6.2-py3-none-any.whl size=8529 sha256=787d3f7ba920c095253cd92220915d5a6ffc4909842d83f384be1300d128c89e
  Stored in directory: /home/weiweia92/.cache/pip/wheels/3f/ac/b2/05c4fc4f364faad4af4130919c228cca4df1d429bcd365fcc1
Successfully built ptflops


In [9]:
import torchvision.models as models

In [11]:
from ptflops import get_model_complexity_info

In [13]:
net = models.vgg16()
flops, params = get_model_complexity_info(net, (3, 224,224), as_strings=True, print_per_layer_stat=True)
print(f'Flops:{flops}')
print('Params: '+params)

VGG(
  138.358 M, 100.000% Params, 15.504 GMac, 100.000% MACs, 
  (features): Sequential(
    14.715 M, 10.635% Params, 15.38 GMac, 99.202% MACs, 
    (0): Conv2d(0.002 M, 0.001% Params, 0.09 GMac, 0.580% MACs, 3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(0.0 M, 0.000% Params, 0.003 GMac, 0.021% MACs, inplace=True)
    (2): Conv2d(0.037 M, 0.027% Params, 1.853 GMac, 11.951% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(0.0 M, 0.000% Params, 0.003 GMac, 0.021% MACs, inplace=True)
    (4): MaxPool2d(0.0 M, 0.000% Params, 0.003 GMac, 0.021% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(0.074 M, 0.053% Params, 0.926 GMac, 5.976% MACs, 64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(0.0 M, 0.000% Params, 0.002 GMac, 0.010% MACs, inplace=True)
    (7): Conv2d(0.148 M, 0.107% Params, 1.851 GMac, 11.941% MACs, 128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)